# Build Extractive QA Pipeline
https://haystack.deepset.ai/tutorials/34_extractive_qa_pipeline  
see similar script (more recent?): `src/haystack/34_extractive_qa_pipeline.py`  

Working

In [1]:
#!pip install haystack-ai accelerate "sentence-transformers>=2.2.0" "datasets>=2.6.1"

In [3]:
from haystack.telemetry import tutorial_running

tutorial_running(34)


In [4]:
from datasets import load_dataset
from haystack import Document
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter

In [5]:
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")

In [6]:
documents = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]
documents

[Document(id=75fd8474f2c88337f7e0dad69eba0f24ba293cb06693fb746ec403df01a1c0c5, content: 'The Colossus of Rhodes (Ancient Greek: ὁ Κολοσσὸς Ῥόδιος, romanized: ho Kolossòs Rhódios Greek: Κολο...', meta: {'url': 'https://en.wikipedia.org/wiki/Colossus_of_Rhodes', '_split_id': 0}),
 Document(id=5e4115a663f0afb5f51c3aba9d04daf6f4fae39031cc55e553e31d7be7f1d734, content: '[6]
 In 653, an Arab force under Muslim general Muawiyah I conquered Rhodes, and according to the Chr...', meta: {'url': 'https://en.wikipedia.org/wiki/Colossus_of_Rhodes', '_split_id': 1}),
 Document(id=c674d039894fc1fdcfef9d3801ae976919c7fbe6b81a189a8ca630cd4e1d7961, content: 'Construction[edit]
 Timeline and map of the Seven Wonders of the Ancient World, including the Colossu...', meta: {'url': 'https://en.wikipedia.org/wiki/Colossus_of_Rhodes', '_split_id': 2}),
 Document(id=6554927b94b6bdbb39d6276775c18233900dbdef5e47b4222130c7b861be8fba, content: 'Philo of Byzantium wrote in De septem mundi miraculis that Chares create

In [7]:
model = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

document_store = InMemoryDocumentStore()

indexing_pipeline = Pipeline()

indexing_pipeline.add_component(instance=SentenceTransformersDocumentEmbedder(model=model), name="embedder")
indexing_pipeline.add_component(instance=DocumentWriter(document_store=document_store), name="writer")
indexing_pipeline.connect("embedder.documents", "writer.documents")

🚅 Components
  - embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - embedder.documents -> writer.documents (List[Document])

In [8]:
indexing_pipeline.run({"documents": documents})

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.24it/s]


{'writer': {'documents_written': 151}}

In [9]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersTextEmbedder


retriever = InMemoryEmbeddingRetriever(document_store=document_store)
reader = ExtractiveReader()
reader.warm_up()

extractive_qa_pipeline = Pipeline()

extractive_qa_pipeline.add_component(instance=SentenceTransformersTextEmbedder(model=model), name="embedder")
extractive_qa_pipeline.add_component(instance=retriever, name="retriever")
extractive_qa_pipeline.add_component(instance=reader, name="reader")

extractive_qa_pipeline.connect("embedder.embedding", "retriever.query_embedding")
extractive_qa_pipeline.connect("retriever.documents", "reader.documents")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - reader: ExtractiveReader
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> reader.documents (List[Document])

In [10]:
query = "Who was Pliny the Elder?"
extractive_qa_pipeline.run(
    data={"embedder": {"text": query}, "retriever": {"top_k": 3}, "reader": {"query": query, "top_k": 2}}
)

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.18it/s]


{'reader': {'answers': [ExtractedAnswer(query='Who was Pliny the Elder?', score=0.8306007385253906, data='Roman writer', document=Document(id=bb2c5f3d2e2e2bf28d599c7b686ab47ba10fbc13c07279e612d8632af81e5d71, content: 'The Roman writer Pliny the Elder, writing in the first century AD, argued that the Great Pyramid had...', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_Giza', '_split_id': 16}, score: 21.66772910369628), context=None, document_offset=ExtractedAnswer.Span(start=4, end=16), context_offset=None, meta={}),
   ExtractedAnswer(query='Who was Pliny the Elder?', score=0.7280879616737366, data='a Roman author', document=Document(id=8910f21f7c0e97792473bcc60a8dcc7f6a90586dbb46b7bf96d28dbfcdc313f4, content: '[21]
   Pliny the Elder (AD 23/24 – 79) was a Roman author, a naturalist and natural philosopher, a nav...', meta: {'url': 'https://en.wikipedia.org/wiki/Colossus_of_Rhodes', '_split_id': 8}, score: 26.85754046335517), context=None, document_offset=ExtractedAnswe